<a href="https://colab.research.google.com/github/MatteoOnger/algo-collusion-mm/blob/main/notebooks/notebook_exp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithmic Collusion in Market Making - Hedge

A notebook testing Hedge agents implementing market-making strategies in the Glosten-Milgrom environment.

## Notebook Initialization

### Colab Environment Setup

In [ ]:
# Do NOT run this cell in local environment - it's intended for Google Colab only.

# Clone GitHub repository
!git clone https://github.com/MatteoOnger/algo-collusion-mm.git

# Set working directory
%cd /content/algo-collusion-mm

# Install dependencies
!pip install --quiet .

### Local Environment Setup

In [1]:
# Do NOT run this cell in Google Colab - it's intended for local Jupyter Notebooks only.

# Autoreload imports
%load_ext autoreload
%autoreload 2

# Select interactive backend for matplotlib
%matplotlib widget

## Main Execution

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import time

import algo_collusion_mm.utils.plots as plots
import algo_collusion_mm.utils.storage as storage

from datetime import datetime
from typing import Dict, List

from algo_collusion_mm.agents import Agent, MakerHedge, NoPassTrader
from algo_collusion_mm.envs import CGMEnv
from algo_collusion_mm.utils.common import get_calvano_collusion_index, get_relative_deviation_competition


plots.DECIMAL_PLACES_VALUES = 2

### Load Agents

In [ ]:
saver = storage.ExperimentStorage(base_path=None)

objects = saver.load_objects(os.path.join('..', 'experiments', 'hedge', 'notebook'))
print(f'Objects loaded: {list(objects.keys())}')

### Run Single Episode

In [3]:
saver = storage.ExperimentStorage(os.path.join('..', 'experiments', 'hedge', 'notebook'))

In [ ]:
n = 10_000          # Number of rounds
k =    100          # Number of windows
w = n // k          # Window size

n_makers = 3        # Number of market makers

nash_reward = 0.05   # Nash reward (single-agent case)
coll_reward = 0.35   # Collusive reward (single-agent case)

counter = 0         # Number of rounds done

# Prices and action space of the market makers
prices =  np.round(np.arange(0.0, 1.0 + 0.2, 0.2), 2)
action_space = np.array([(ask, bid) for ask in prices for bid in prices if (ask  > bid)])

# Create agents
makers: List[MakerHedge] = [
    MakerHedge(epsilon=MakerHedge.compute_epsilon(len(action_space), n), scale_rewards=lambda r: (r / 0.3), action_space=action_space, name='maker_i_0'),
    MakerHedge(epsilon=MakerHedge.compute_epsilon(len(action_space), n), scale_rewards=lambda r: (r / 0.3), action_space=action_space, name='maker_i_1'),    
]
traders : List[NoPassTrader] = [
    NoPassTrader(name='trader_0', tie_breaker='rand'),
]
agents: Dict[str, Agent] = {agent.name: agent for agent in makers + traders}

# Create env
env = CGMEnv(
    generate_vt = lambda: 0.5,
    n_rounds = n,
    makers = makers,
    traders = traders,
    info_level = 'partial'
)

start_time = time.time()
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Started at {current_time}')

_, info = env.reset()
for next_player in env.agent_iter():
    action = next_player.act(env.observe(next_player))
    _, rewards, _, _, infos = env.step(action)

    if infos['round_finished']:
        if counter % 10_000 == 0:
            print(f'Running round {counter} ...')

        for agent in env.possible_agents:
            # Save the current belif of the agent
            if agent in env.makers and counter % (k//5) == 0:
                agent.history.record_extra(agent.probs.copy())
            
            agent.update(rewards[agent.name], infos[agent.name])
        counter += 1

end_time = time.time()
execution_time = end_time - start_time
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f'Done at {current_time} | Execution time: {execution_time:.2f} seconds')

# Compute calvano collusion idex per window and agent
cci = get_calvano_collusion_index(
    np.array([maker.history.get_rewards() for maker in makers]),
    nash_reward = nash_reward,
    coll_reward = coll_reward,
    window_size = w,
)

# Compute relative deviation from competition index per window and agent
rdc = get_relative_deviation_competition(
    np.array([maker.history.get_rewards() for maker in makers]),
    nash_reward = nash_reward,
    window_size = w,
)

# Collect info
info = {
    'parmas' : {
        'n_rounds' : n,
        'window_size' : w,
        'action_space' : str(action_space).replace('\n', ','),
        'tie_breaker' : [trader.tie_breaker for trader in traders],
        'epsilon' : [maker.epsilon for maker in makers],
        'seed' : [agent._seed for agent in agents.values()],
        'agent_type' : [agent.__class__.__name__ for agent in agents.values()],
    },
    'freq_actions' : {
        0 : {maker.name : str(maker.history.compute_freqs(slice(0, w))).replace('\n', '') for maker in makers},
        k : {maker.name : str(maker.history.compute_freqs(slice(-w, None))).replace('\n', '') for maker in makers},
        'global' : {maker.name : str(maker.history.compute_freqs()).replace('\n', '') for maker in makers}
    },
    'most_common_action' : {
        0 : {maker.name : str(maker.history.compute_most_common(slice(0, w))) for maker in makers},
        k : {maker.name : str(maker.history.compute_most_common(slice(-w, None))) for maker in makers},
        'global' : {maker.name : str(maker.history.compute_most_common()) for maker in makers}
    },
    'cumulative_rewards' : {
        0 : {name : round(float(agent.history.get_rewards(slice(0, w)).sum()), 3) for name, agent in agents.items()},
        k : {name : round(float(agent.history.get_rewards(slice(-w, None)).sum()), 3) for name, agent in agents.items()},
        'global' : env.cumulative_rewards
    },
    'cci' : {
        0  : {maker.name : round(float(cci[idx, 0]), 3) for idx, maker in enumerate(makers)},
        k  : {maker.name : round(float(cci[idx, -1]), 3) for idx, maker in enumerate(makers)},
        'global' : {maker.name : round(float(cci[idx, :].mean()), 3) for idx, maker in enumerate(makers)},
    },
    'rdc' : {
        0  : {maker.name : round(float(rdc[idx, 0]), 3) for idx, maker in enumerate(makers)},
        k  : {maker.name : round(float(rdc[idx, -1]), 3) for idx, maker in enumerate(makers)},
        'global' : {maker.name : round(float(rdc[idx, :].mean()), 3) for idx, maker in enumerate(makers)},
    }
}

# Plot figure
fig = plots.plot_all(
    window_size = w,
    makers = makers,
    makers_action_values_name = 'probs',
    cci = cci,
    nash_reward = nash_reward,
    coll_reward = coll_reward,
    title = 'Hedge Makers Summary Plots'
)

# Save results
dir =  saver.save_episode([env] + list(agents.values()), fig, info)

print(json.dumps(info, indent=2))
display(fig)
print(dir)

### Additional Plots

In [ ]:
plt.close()

plots.plot_maker_action_values_evolution_sc(
    agents['maker_i_0'],
    curr_idx = 0,
    next_idx = 100
)

plt.show()